<a href="https://colab.research.google.com/github/aq1048576/COS217_A2/blob/master/pave_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/')


Mounted at /content/drive/


In [41]:
# copy and pasted online code for finding circle using Hough Transform method

def find_circle(image):
  
  img = cv.imread(image,0)
  img = cv.medianBlur(img,5)
  cimg = cv.cvtColor(img,cv.COLOR_GRAY2BGR)
  circles = cv.HoughCircles(img,cv.HOUGH_GRADIENT,1,20,
                              param1=50,param2=30,minRadius=0,maxRadius=0)
  if circles is None:
    return 0,0,0
  circles = np.uint16(np.around(circles))
  for i in circles[0,:]:
    return i[0], i[1], i[2]

(0, 0, 0)

In [70]:
# find the distance from the boat to the buoy using online formula

def find_dist(name, radius):
  if (radius == 0):
    return -1
  img = cv.imread(name,0)
  h,w = img.shape
  # goPro stats from web
  focal = 16.5
  sensor = 11.04
  # Wikipedia says 1.5-12 meters, I chose 8 meters
  real_buoy_height = 8000
  image_height = h
  image_buoy_height = 2*radius
  # using formula from web
  # focal * real * image_height// (sensor * image buoy height)
  dist = (focal * real_buoy_height * image_height)//(sensor * image_buoy_height)
  return dist

In [81]:
# looks like all of the images have two horizontal lines, all at the same location. this code snippet uses a random image
# to find the y coordinate of the lower line, and if the top of the buoy is higher than the lower line, then we reject the image
# line location ends up around 399 pixels, horizon is horizontal with a slight tilt of 1.57 degrees, which we disregard

def find_horizon(name):
  img = cv2.imread(name)
  
  # Convert the img to grayscale
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  
  # Apply edge detection method on the image
  edges = cv2.Canny(gray,50,150,apertureSize = 3)
    
  # This returns an array of r and theta values
  lines = cv2.HoughLines(edges,1,np.pi/180, 200)
  print(lines)
    
find_horizon('/content/drive/MyDrive/pave_intro/images/lake03.jpeg')
      

[[[199.          1.5707964]]

 [[399.          1.5707964]]]


In [85]:
# checks if top of buoy in image is in the trees or sky

def check_bad(radius, center_height):
  if ((center_height-radius)<= 399):
    return True

In [84]:
# main driving code

# places the names of the files into a list
namelist = []
for i in range(10):
  for j in range (10):
    filename = '/content/drive/MyDrive/pave_intro/images/lake'+str(i)+str(j)+'.jpeg'
    namelist.append(filename)
radius = np.array([])
center = np.zeros([100,2])
distance = np.array([])

#iterates through the list, if the buoy is nonexistent or not in the water, we append -1 values to the center, radius, and distance lists
for i in range(100):
  name = namelist[i]
  x, y, r = find_circle(name)
  if (r==0 or check_bad(r, y)):
    center[i] = -1
    radius = np.append(radius, -1)
    distance = np.append(distance, -1)
  else: 
    center[i][0] = x
    center[i][1] = y
    radius = np.append(radius, r)
    dist = int(find_dist(name, r))
    distance = np.append(distance, dist)

print('center', center)
print('radius', radius)
print('distance', distance)


center [[ 6.040e+02  7.200e+02]
 [ 1.592e+03  1.650e+03]
 [ 3.600e+02  4.300e+02]
 [-1.000e+00 -1.000e+00]
 [ 5.520e+02  1.742e+03]
 [-1.000e+00 -1.000e+00]
 [-1.000e+00 -1.000e+00]
 [ 2.292e+03  1.356e+03]
 [ 1.294e+03  1.014e+03]
 [-1.000e+00 -1.000e+00]
 [ 2.218e+03  1.680e+03]
 [-1.000e+00 -1.000e+00]
 [ 1.072e+03  1.672e+03]
 [ 1.544e+03  5.340e+02]
 [ 2.280e+03  1.826e+03]
 [ 7.020e+02  1.702e+03]
 [ 1.806e+03  8.200e+02]
 [ 2.656e+03  1.164e+03]
 [-1.000e+00 -1.000e+00]
 [-1.000e+00 -1.000e+00]
 [ 6.160e+02  8.400e+02]
 [ 1.754e+03  1.400e+03]
 [ 2.058e+03  8.260e+02]
 [-1.000e+00 -1.000e+00]
 [-1.000e+00 -1.000e+00]
 [ 1.456e+03  1.306e+03]
 [ 1.536e+03  8.120e+02]
 [ 4.340e+02  1.704e+03]
 [ 7.840e+02  1.282e+03]
 [ 1.026e+03  1.906e+03]
 [-1.000e+00 -1.000e+00]
 [ 2.424e+03  6.520e+02]
 [ 8.600e+01  1.224e+03]
 [-1.000e+00 -1.000e+00]
 [-1.000e+00 -1.000e+00]
 [ 4.380e+02  1.638e+03]
 [-1.000e+00 -1.000e+00]
 [ 1.772e+03  5.100e+02]
 [ 1.234e+03  1.882e+03]
 [ 3.800e+01  1.82